# Домашнее задание

В этом домашнем задании вам нужно будет по различным характеристикам спрогнозировать, будет ли завтра в Австралии дождь или нет. Для прогноза будем использовать AutoML!

## Импорт библиотек и подготовка данных

In [1]:
!pip install tpot -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 717.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
from tpot import TPOTClassifier

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/aiedu-courses/stepik_applied_tasks/main/datasets/weatherAUS.csv")

df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


### Вопрос 1

В скольких столбцах таблицы есть пропуски?

In [9]:
df.shape

(145460, 23)

In [8]:
df.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

### Вопрос 2

Удалите строки таблицы, где пропущено значение целевой переменной (`RainTomorrow`). Сколько строк осталось в таблице?

In [3]:
df = df[~df.RainTomorrow.isna()]
df.shape

(142193, 23)

Удалите столбцы таблицы, где содержится более 20000 пропусков.

In [42]:
df.columns[df.isna().sum() > 20000]

Index(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], dtype='object')

In [4]:
df.drop(columns = df.columns[df.isna().sum() > 20000], inplace=True)

Заполните пропуски:
* в числовых столбцах - медианным значением
* в категориальных столбцах - новой категорией "Empty"

In [25]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No,No


In [24]:
df.isna().sum()

Date                 0
Location             0
MinTemp            637
MaxTemp            322
Rainfall          1406
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Temp9am            904
Temp3pm           2726
RainToday         1406
RainTomorrow         0
dtype: int64

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   WindGustDir    132863 non-null  object 
 6   WindGustSpeed  132923 non-null  float64
 7   WindDir9am     132180 non-null  object 
 8   WindDir3pm     138415 non-null  object 
 9   WindSpeed9am   140845 non-null  float64
 10  WindSpeed3pm   139563 non-null  float64
 11  Humidity9am    140419 non-null  float64
 12  Humidity3pm    138583 non-null  float64
 13  Pressure9am    128179 non-null  float64
 14  Pressure3pm    128212 non-null  float64
 15  Temp9am        141289 non-null  float64
 16  Temp3pm        139467 non-null  float64
 17  RainToday      140787 non-nul

In [5]:
df[df.select_dtypes('object').columns] = df[df.select_dtypes('object').columns].fillna(value='Empty')

In [38]:
df[df.select_dtypes('float64').columns]

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7
...,...,...,...,...,...,...,...,...,...,...,...,...
145454,3.5,21.8,0.0,31.0,15.0,13.0,59.0,27.0,1024.7,1021.2,9.4,20.9
145455,2.8,23.4,0.0,31.0,13.0,11.0,51.0,24.0,1024.6,1020.3,10.1,22.4
145456,3.6,25.3,0.0,22.0,13.0,9.0,56.0,21.0,1023.5,1019.1,10.9,24.5
145457,5.4,26.9,0.0,37.0,9.0,9.0,53.0,24.0,1021.0,1016.8,12.5,26.1


In [6]:
df[df.select_dtypes('float64').columns] = df[df.select_dtypes('float64').columns].fillna(value=df[df.select_dtypes('float64').columns].median())

In [46]:
df.isna().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [47]:
# replacement= {'column_A': 0, 'column_B': -999, 'column_C': -99999}
# df.fillna(value=replacement)

### Вопрос 3

Чему равна медиана в столбце `MinTemp` после заполнения пропусков? Ответ округлите до целого числа.

In [9]:
df.MinTemp.median()

12.0

## Обучение baseline-модели

Выделим матрицу объект-признак и целевую переменную

In [7]:
X = df.drop('RainTomorrow', axis=1)
y = df['RainTomorrow']

Разобъем данные на тренировочную и тестовую части

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

y_train.shape

(106644,)

### Вопрос 4

Для baseline-модели будем использовать только числовые признаки (они имеют тип float).

Создайте матрицы `X_train_num` и `X_test_num`, состоящие только из числовых признаков.

Сколько столбцов в каждой из этих матриц?

In [9]:
X_train_num = X_train[X_train.select_dtypes('float64').columns]
X_test_num = X_test[X_test.select_dtypes('float64').columns]

X_train_num.shape

(106644, 12)

Обучим на тренировочных данных пайплайн, состоящий из масштабирования данных, а затем применения логистической регрессии.

In [52]:
lr_pipe = Pipeline([('scaler', StandardScaler()),
                    ('model', LogisticRegression())])

lr_pipe.fit(X_train_num, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', LogisticRegression())])

### Вопрос 5

Сделайте прогноз на тестовых данных и вычислите значение метрики roc-auc. Чему равен roc-auc, округленный до тысячных?

In [57]:
y_pred

array([[0.17464954, 0.82535046],
       [0.59149318, 0.40850682],
       [0.97469293, 0.02530707],
       ...,
       [0.96179802, 0.03820198],
       [0.28959878, 0.71040122],
       [0.53578866, 0.46421134]])

In [58]:
y_pred = lr_pipe.predict_proba(X_test_num)[:,1]
roc_auc_score(y_test, y_pred)

0.8473240633372361

## AutoML

В этой секции применим AutoML-фреймворки для получения прогнозов.

### TPOT

Начнем с фреймворка TPOT.

Объявите классификатор TPOTClassifier со следующими гиперпараметрами:
* `generations = 2`
* `population_size = 5`
* `verbosity = 2`
* `random_state = 42`

Обучите классификатор на тренировочных данных (`X_train_num, y_train`).

In [59]:
tpot = TPOTClassifier(generations = 2, population_size = 5, verbosity = 2, random_state = 42).fit(X_train_num, y_train)

Optimization Progress:   0%|          | 0/15 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8483458784898765

Generation 2 - Current best internal CV score: 0.8483458784898765

Best pipeline: RandomForestClassifier(MinMaxScaler(input_matrix), bootstrap=True, criterion=gini, max_features=0.2, min_samples_leaf=8, min_samples_split=4, n_estimators=100)


### Вопрос 6

Сделайте прогноз на тестовых данных и вычислите значение метрики roc-auc. Чему равен roc-auc, округленный до тысячных?

In [60]:
y_tpot = tpot.predict_proba(X_test_num)[:,1]

In [61]:
roc_auc_score(y_test, y_tpot)

0.8618761848514245

### Auto-Sklearn

Обучение AutoSklearn-моделей может быть ОЧЕНЬ долгим!

In [1]:
# 1. uninstall all affected packages
!pip uninstall -y Cython scipy pyparsing scikit_learn imbalanced-learn mlxtend yellowbrick

# 2. install packages to be downgraded
!pip install Cython==0.29.36 scipy==1.9 pyparsing==2.4

Found existing installation: Cython 0.29.36
Uninstalling Cython-0.29.36:
  Successfully uninstalled Cython-0.29.36
Found existing installation: scipy 1.9.0
Uninstalling scipy-1.9.0:
  Successfully uninstalled scipy-1.9.0
Found existing installation: pyparsing 2.4.0
Uninstalling pyparsing-2.4.0:
  Successfully uninstalled pyparsing-2.4.0
Found existing installation: scikit-learn 0.24.2
Uninstalling scikit-learn-0.24.2:
  Successfully uninstalled scikit-learn-0.24.2
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Using cached scipy-1.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.9 MB)
  Using cached pyparsing-2.4.0-py2.py3-none-any.whl (62 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tpot 0.12.1 requires scikit-learn>=0.22.0, which is not installed.
auto-sklearn 

**Важно!** Перезапустите среду выполнения.

In [1]:
# 3. install older scikit-learn disregarding its dependencies
!pip install scikit-learn==0.24.2 --no-build-isolation

# 4. finally install auto-sklearn
!pip install auto-sklearn

# 5. then, try loading the package repeatedly until trash in its dependencies are clean
import autosklearn

  Using cached scikit_learn-0.24.2-cp310-cp310-linux_x86_64.whl
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-core 1.0.0 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 0.24.2 which is incompatible.
autogluon-features 1.0.0 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 0.24.2 which is incompatible.
autogluon-multimodal 1.0.0 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 0.24.2 which is incompatible.
autogluon-tabular 1.0.0 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 0.24.2 which is incompatible.
bigframes 0.16.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.1.4 which is incompatible.
bigframes 0.16.0 requires scikit-learn>=1.2.2, but you have scikit-learn 0.24.2 which is incompatible.


**Важно!** Перезапустите среду выполнения. Здесь придется заново перезапустить все ячейки с начала ноутбука (кроме ячеек с установкой и импортом TPOT).

Обучите модель AutoSklearnClassifier на тренировочных данных (`X_train_num, y_train`) с гиперпараметрами:
* `tmp_folder='autosklearn_classification_example_tmp'`


In [11]:
from autosklearn.classification import AutoSklearnClassifier

autosk = AutoSklearnClassifier(tmp_folder='autosklearn_classification_example_tmp').fit(X_train_num, y_train)

y_autosk = autosk.predict_proba(X_test_num)[:,1]

### Вопрос 7

Сделайте прогноз на тестовых данных и вычислите значение метрики roc-auc. Чему равен roc-auc, округленный до тысячных?

In [12]:
roc_auc_score(y_test, y_autosk)

0.8677892414109007

AutoSklearn умеет кодировать категориальные признаки самостоятельно. Попробуем это сделать.

In [13]:
X_train.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Temp9am          float64
Temp3pm          float64
RainToday         object
dtype: object

Обучим модель на всех тренировочных данных (не только на числовых).

Для скорости обучайте модель только на первых 100 объектах тренировочной выборки (все равно это будет долго).

Сначала необходимо создать список типов колонок - в таком порядке, в котором они идут в таблице `X_train`.
Например, если сначала идет 3 числовых колонки, затем 1 категориальная, а затем 9 числовых, то пишем:


In [11]:
from autosklearn.classification import AutoSklearnClassifier

In [26]:
# feature_types = (['numerical'] * 3) + ['categorical'] + (['numerical'] * 9)

feature_types = (['categorical'] * 2) + (['numerical'] * 3) + ['categorical'] + ['numerical'] + (['categorical'] * 2) + (['numerical'] * 8) + ['categorical']

autosk_cat = AutoSklearnClassifier(
                              tmp_folder='autosklearn_classification_example_2_tmp')

autosk_cat.fit(X_train.iloc[:100,:], y_train[:100])
y_autosk_cat = autosk_cat.predict_proba(X_test)[:,1]

/usr/local/lib/python3.10/dist-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column Date has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column Location has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column WindGustDir has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column WindDir9am has generic type object. Autosklearn will treat this column as string. Please ensure t

[WARNING] [2023-12-15 15:26:34,922:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-12-15 15:26:36,344:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-12-15 15:26:37,875:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-12-15 15:26:39,450:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-12-15 15:26:42,168:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1


### Вопрос 8

Сделайте прогноз последней обученной модели на тестовых данных и вычислите значение метрики roc-auc. Чему равен roc-auc, округленный до тысячных?

Напишите ответ в комментариях, если дождались результата!

In [27]:
roc_auc_score(y_test, y_autosk_cat)

0.7829359635492295

## AutoGluon

Наконец, попробуем применить AutoGluon.

**Важно!** Перед установкой AutoGluon рекомендуется заново запустить ноутбук и ячейки, связанные с загрузкой и обработкой данных.

In [1]:
!pip install autogluon

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

Создайте объект `train_data`, в который скопируйте `X_train`, а также добавьте столбец `target` - в него скопируйте `y_train`.

Обучите `TabularPrediction` на тренировочных данных (`train_data`).

In [13]:
X_train['target'] = y_train

In [14]:
X_train

,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,target
105663,2009-04-12,Woomera,14.9,30.3,0.0,S,33.0,ESE,SW,15.0,11.0,19.0,12.0,1021.1,1017.8,22.2,29.7,No,No
113689,2014-12-08,Witchcliffe,14.6,21.5,0.2,SSE,46.0,S,SSE,26.0,28.0,65.0,57.0,1012.6,1013.5,17.5,18.6,No,No
126081,2015-06-06,SalmonGums,9.0,23.7,0.0,W,28.0,WNW,W,11.0,15.0,59.0,45.0,1017.6,1015.2,14.6,23.1,No,No
110347,2014-01-09,Albany,15.3,24.0,0.0,Empty,39.0,SSW,SSW,4.0,15.0,63.0,82.0,1018.1,1016.7,21.8,21.8,No,No
72477,2014-12-14,Mildura,17.3,37.5,0.0,N,39.0,NE,NW,9.0,15.0,26.0,12.0,1009.6,1006.2,23.8,35.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112920,2012-09-01,Witchcliffe,9.0,18.9,12.6,W,43.0,WNW,W,20.0,15.0,70.0,52.0,1017.5,1017.7,15.8,17.6,Yes,Yes
122810,2014-09-09,Perth,13.1,19.9,3.0,SW,39.0,SW,W,2.0,13.0,90.0,55.0,1018.1,1018.2,15.4,19.1,Yes,No
106280,2010-12-20,Woomera,10.8,25.5,0.0,SSW,44.0,SSW,SSE,26.0,15.0,41.0,21.0,1015.7,1012.6,15.3,23.1,No,No
135107,2014-11-05,Launceston,11.3,21.4,0.0,ESE,39.0,SSE,SE,19.0,22.0,75.0,79.0,1017.6,1015.2,15.8,15.7,No,Yes


In [15]:
train_data = TabularDataset(X_train)

In [16]:
save_path = 'agModels-predictClass'
predictor = TabularPredictor(label='target', path=save_path).fit(train_data)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating Syst

### Вопрос 9

Сделайте прогноз последней обученной модели на тестовых данных и вычислите значение метрики roc-auc. Чему равен roc-auc, округленный до тысячных?

In [17]:
autogluon_probs = predictor.predict_proba(X_test) # получаем два столбца - 'No', 'Yes'. Для расчета roc-auc вам нужен столбец 'Yes'

roc_auc_score(y_test, autogluon_probs.Yes)

0.9001350544157101

### Oboe и эксперименты с фрейморками

Эта часть бонусная. Попробуйте

а) Применить фреймворк Oboe/TensorOboe для решения поставленной задачи

б) Подобрать гиперпараметры у остальных фреймворков для улучшения качества прогноза

в) Обогнать AutoML собственным пайплайном!

In [28]:
!pip install oboe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 9.6 MB/s eta 0:00:00
  Created wheel for OpenML: filename=openml-0.14.1-py3-none-any.whl size=146941 sha256=540c8cac3ddb4001d654c69f0704b908fd7f128c45958890004ee1e1bc591a25
  Stored in directory: /root/.cache/pip/wheels/75/bc/fd/739778254a2881ef96b139d0aaf60c6d4f9130bb1459b48f10
Successfully built OpenML


In [29]:
from oboe import AutoLearner, error
import warnings
warnings.filterwarnings('ignore')

method = 'tensoroboe'
problem_type = 'classification'

In [48]:
# initialize the autolearner class
m = AutoLearner(p_type=problem_type, runtime_limit=10, method=method, verbose=True)

rank for EM-Tucker imputation: (20, 4, 2, 2, 8, 20)
shape of the error tensor: (551, 4, 2, 2, 8, 183)
Loading latent factors from storage ...
Loading saved runtime predictors ...


In [32]:
X_train.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday
105663,2009-04-12,Woomera,14.9,30.3,0.0,S,33.0,ESE,SW,15.0,11.0,19.0,12.0,1021.1,1017.8,22.2,29.7,No
113689,2014-12-08,Witchcliffe,14.6,21.5,0.2,SSE,46.0,S,SSE,26.0,28.0,65.0,57.0,1012.6,1013.5,17.5,18.6,No
126081,2015-06-06,SalmonGums,9.0,23.7,0.0,W,28.0,WNW,W,11.0,15.0,59.0,45.0,1017.6,1015.2,14.6,23.1,No
110347,2014-01-09,Albany,15.3,24.0,0.0,Empty,39.0,SSW,SSW,4.0,15.0,63.0,82.0,1018.1,1016.7,21.8,21.8,No
72477,2014-12-14,Mildura,17.3,37.5,0.0,N,39.0,NE,NW,9.0,15.0,26.0,12.0,1009.6,1006.2,23.8,35.7,No


In [34]:
X_train.isna().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Temp9am          0
Temp3pm          0
RainToday        0
dtype: int64

In [36]:
cols = X_test.select_dtypes('object').columns

In [39]:
cols.to_list()

['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']

In [45]:
np.array(X_train)

array([['2009-04-12', 'Woomera', 14.9, ..., 22.2, 29.7, 'No'],
       ['2014-12-08', 'Witchcliffe', 14.6, ..., 17.5, 18.6, 'No'],
       ['2015-06-06', 'SalmonGums', 9.0, ..., 14.6, 23.1, 'No'],
       ...,
       ['2010-12-20', 'Woomera', 10.8, ..., 15.3, 23.1, 'No'],
       ['2014-11-05', 'Launceston', 11.3, ..., 15.8, 15.7, 'No'],
       ['2012-02-07', 'SalmonGums', 10.3, ..., 19.7, 26.7, 'No']],
      dtype=object)

In [46]:
m.fit(np.array(X_train), np.array(y_train), categorical=cols) # TensorOboe принимает на вход список категориальных признаков для кодирования


Shape of training dataset: 106644 data points, 18 features
dataset too skewed in having too many data points; sampling data points with sampling ratio 0.01687858669967368

Training dataset resampled 
Shape of resampled training dataset: 1800 data points, 18 features
Splitting training set into training and validation ..
Predicting pipeline running time ..
runtime limit of initial round: 32.0 seconds
fitting and kfold_fit_validating the best-on-average pipeline


TypeError: ignored

In [50]:
m.fit(np.array(X_train_num), np.array(y_train), categorical=None)


Shape of training dataset: 106644 data points, 12 features
dataset too skewed in having too many data points; sampling data points with sampling ratio 0.011252391133115788

Training dataset resampled 
Shape of resampled training dataset: 1200 data points, 12 features
Splitting training set into training and validation ..
Predicting pipeline running time ..
runtime limit of initial round: 32.0 seconds
fitting and kfold_fit_validating the best-on-average pipeline
Pipeline fitting completed.
Fitted an ensemble with size 1
having a capped running time of 32 seconds
Fitted an ensemble with size 1


ValueError: ignored

In [42]:
X_train.shape, y_train.shape

((106644, 18), (106644,))